### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
pip install oci psycopg2-binary sentence-transformers pandas pyarrow tqdm weasyprint


In [ ]:
# --- 1. Imports ---
import os
import oci
import pandas as pd
from tqdm import tqdm
import psycopg2
from sentence_transformers import SentenceTransformer

# --- 2. Configuration ---

# OCI Object Storage
BUCKET_NAME = "aus-legal-corpus"
OBJECT_PREFIX = ""  # Assuming root directory of the bucket
DOWNLOAD_DIR = "./data"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# PostgreSQL
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "",
    "host": "10.150.2.103",
    "port": "5432"
}

# OCI Config
oci_config = {
    "user": "ocid1.user.oc1..aaq",
    "key_file": "./data/oci_api_key.pem",
    "fingerprint": "de:d6",
    "tenancy": "ocid1.tenancy.oc1..aua",
    "region": "us-sanjose-1"
}

# --- 3. Connect to OCI and Download Parquet Files ---
object_storage = oci.object_storage.ObjectStorageClient(oci_config)
namespace = object_storage.get_namespace().data

print("🔍 Listing objects in bucket...")
objects = object_storage.list_objects(namespace, BUCKET_NAME, prefix=OBJECT_PREFIX).data.objects
parquet_files = [obj.name for obj in objects if obj.name.endswith(".parquet")]

for obj_name in parquet_files:
    local_file = os.path.join(DOWNLOAD_DIR, os.path.basename(obj_name))
    if not os.path.exists(local_file):
        print(f"⬇️ Downloading {obj_name} ...")
        with open(local_file, 'wb') as f:
            response = object_storage.get_object(namespace, BUCKET_NAME, obj_name)
            for chunk in response.data.raw.stream(1024 * 1024, decode_content=False):
                f.write(chunk)
print("✅ All Parquet files downloaded.")

# --- 4. Load Embedding Model ---
model = SentenceTransformer("BAAI/bge-small-en-v1.5")
print("✅ Embedding model loaded (384 dimensions).")

# --- 5. Connect to PostgreSQL ---
conn = psycopg2.connect(**DB_CONFIG)
conn.autocommit = True
cursor = conn.cursor()

# --- 6. Create Table with Vector Extension ---
cursor.execute("""
CREATE EXTENSION IF NOT EXISTS vector;

CREATE TABLE IF NOT EXISTS legal_docs (
    id SERIAL PRIMARY KEY,
    content TEXT,
    embedding VECTOR(384)
);
""")
print("✅ Table ready.")

# --- 7. Insert Helper with Error Handling ---
def insert_batch(batch):
    if not batch:
        print("⚠️  Empty batch, skipping insert.")
        return
    try:
        args_str = ",".join(cursor.mogrify("(%s, %s)", x).decode("utf-8") for x in batch)
        cursor.execute("INSERT INTO legal_docs (content, embedding) VALUES " + args_str)
        print(f"✅ Inserted {len(batch)} records.")
    except Exception as e:
        print(f"❌ Error during batch insert: {e}")

# --- 8. Process and Vectorize Each Parquet File ---
BATCH_SIZE = 500
local_files = sorted([f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".parquet")])
total_inserted = 0

for file in local_files:
    file_path = os.path.join(DOWNLOAD_DIR, file)
    print(f"\n🔄 Processing {file} ...")
    
    try:
        df = pd.read_parquet(file_path)
        print(f"📄 Columns found: {list(df.columns)}")
        print(f"🔢 Rows in file: {len(df)}")
        
        if "text" not in df.columns:
            print(f"⚠️  'text' column not found in {file}, skipping.")
            continue

        df = df[df["text"].notna() & (df["text"].str.strip() != "")]
        print(f"🧹 Rows with non-empty 'text': {len(df)}")

        batch = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Embedding & Inserting"):
            try:
                text = row["text"]
                embedding = model.encode(text).tolist()
                if len(embedding) != 384:
                    print("❌ Skipping due to unexpected vector length.")
                    continue
                batch.append((text, embedding))
                if len(batch) >= BATCH_SIZE:
                    insert_batch(batch)
                    total_inserted += len(batch)
                    batch = []
            except Exception as e:
                print(f"⚠️  Skipping row due to error: {e}")
        
        if batch:
            insert_batch(batch)
            total_inserted += len(batch)

    except Exception as e:
        print(f"❌ Failed to process {file}: {e}")

print(f"\n✅ Total records inserted: {total_inserted}")

# --- 9. Create Vector Index for Cosine Similarity ---
print("⚙️ Creating vector index for cosine similarity...")
cursor.execute("""
CREATE INDEX IF NOT EXISTS legal_docs_cosine_idx
ON legal_docs USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);
""")
cursor.execute("ANALYZE legal_docs;")
print("✅ Vector index created.")

# --- 10. Final Table Check ---
cursor.execute("SELECT COUNT(*) FROM legal_docs;")
print("📊 Total rows in legal_docs:", cursor.fetchone()[0])


🔍 Listing objects in bucket...
✅ All Parquet files downloaded.
✅ Embedding model loaded (384 dimensions).
✅ Table ready.

🔄 Processing 0000.parquet ...
📄 Columns found: ['version_id', 'type', 'jurisdiction', 'source', 'mime', 'date', 'citation', 'url', 'when_scraped', 'text']
🔢 Rows in file: 11024
🧹 Rows with non-empty 'text': 11024


Embedding & Inserting:   5%|▍         | 504/11024 [00:16<19:36,  8.94it/s]

✅ Inserted 500 records.


Embedding & Inserting:   9%|▉         | 1008/11024 [00:33<13:38, 12.23it/s]

✅ Inserted 500 records.


...

✅ Inserted 500 records.


Embedding & Inserting:  97%|█████████▋| 11506/11848 [05:53<00:22, 15.27it/s]

✅ Inserted 500 records.


Embedding & Inserting: 100%|██████████| 11848/11848 [06:02<00:00, 32.67it/s]


✅ Inserted 348 records.

✅ Total records inserted: 118346
⚙️ Creating vector index for cosine similarity...
✅ Vector index created.
📊 Total rows in legal_docs: 118346
